In [4]:
search_for = 123
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004289150238037109
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 350465005
type: [1, 1, 1, 1, 123] 123
cases of this type: 228886641
100000 16.5137192173875
200000 25.560256235858226
300000 25.641626825526266
400000 25.308929110321802
500000 27.78248499837037
600000 24.751819330967287
700000 24.368993888795096
800000 24.624622274252694
900000 23.435131149587622
1000000 10.680231034442992
1100000 23.98301840501423
1200000 24.252925374089777
1300000 24.15296973104777
1400000 24.980023416585432
1500000 25.16978383362975
1600000 24.434161207869224
1700000 24.677616152418032
1800000 25.03423653894913
1900000 10.032742676519215
2000000 25.254075197771435
2100000 30.37939707149441
2200000 29.476370067615438
2300000 29.560756476152005
2400000 30.123258591381894
2500000 28.513510038894793
2600000 28.20047827783734
2700000 28.24026614995295
2800000 17.947371726262958
2900000 22.56042073106009
3000000 2

29400000 24.05224414830824
29500000 23.97469161264984
29600000 12.18360034036125
29700000 23.445325692706913
29800000 24.43538835453826
29900000 24.108609507200416
30000000 21.86965598636947
30100000 12.955238441920399
30200000 24.300466395881795
30300000 23.69463362103928
30400000 23.1848911877368
30500000 22.19550988300976
30600000 20.7881431636021
30700000 15.842825997606816
30800000 14.74288732911008
30900000 20.691682912324904
31000000 21.785013642932764
31100000 23.185813629872015
31200000 22.73978258329887
31300000 23.903055533113886
31400000 15.011568509035769
31500000 20.44666526867791
31600000 23.835812789333584
31700000 24.367635099842815
31800000 23.537716509259972
31900000 17.723913840014546
32000000 18.31036669701865
32100000 24.52385382015951
32200000 22.983687591896633
32300000 23.663811639225692
32400000 21.794104069865785
32500000 21.065844191969422
32600000 8.804580552582175
32700000 22.271771815649547
32800000 21.133235538287632
32900000 23.292267632052102
33000000 

59100000 11.140278477210448
59200000 20.133516576185286
59300000 20.548829706552798
59400000 20.959164966365528
59500000 21.636066749040843
59600000 22.32644661237762
59700000 21.576459213965652
59800000 21.314081658219635
59900000 20.61679601726332
60000000 21.344809427637443
60100000 9.12505312617584
60200000 19.491387955207255
60300000 18.84912840579737
60400000 19.234394061117378
60500000 9.720907613311125
60600000 17.207922887178267
60700000 19.395801007795495
60800000 18.832050067019633
60900000 16.75079472416889
61000000 13.477697688415072
61100000 19.996597406543646
61200000 20.665458053746814
61300000 20.825409786019833
61400000 21.26748296201908
61500000 22.02023429262605
61600000 21.4481447568447
61700000 21.544340361661874
61800000 20.226947069893786
61900000 17.48604342150266
62000000 12.30658688834955
62100000 18.909871220849272
62200000 20.063944411741854
62300000 19.743217506239972
62400000 7.639700246367167
62500000 19.490511851734034
62600000 18.83863166482582
6270000

88900000 18.053289513522156
89000000 17.682548480179495
89100000 18.062437189356245
89200000 19.011301932803512
89300000 19.713778108153136
89400000 19.850007170381144
89500000 19.450801707250477
89600000 18.523132933882714
89700000 17.909713132217476
89800000 17.773983605090223
89900000 18.330599332442155
90000000 16.855326260177673
90100000 8.684800369084716
90200000 17.05677778496788
90300000 6.535542221221908
90400000 17.9153334310295
90500000 9.072553916725616
90600000 17.587329988053906
90700000 17.955272264383026
90800000 17.48972187500053
90900000 17.394511115931405
91000000 18.073427872435303
91100000 19.1631307682935
91200000 19.48494878464803
91300000 20.21379931270353
91400000 18.928585991450362
91500000 17.445987922083898
91600000 17.84233696209938
91700000 18.177924361868357
91800000 17.78632352035955
91900000 17.401747572049437
92000000 8.836465190058028
92100000 14.06995236266423
92200000 10.881996386871478
92300000 12.241319493835976
92400000 14.207093514407063
9250000

118000000 1.7936747485021947
118100000 1.8028308634265198
118200000 1.7952474095982378
118300000 1.7673733041041495
118400000 1.7813657275987929
118500000 1.7904819962148404
118600000 1.8755446590480107
118700000 1.7996684024858574
118800000 1.810096113244262
118900000 1.7887906769382953
119000000 1.8903532230609557
119100000 2.6282916491890256
119200000 23.534078285333365
119300000 22.191344110779976
119400000 21.98910617273736
119500000 21.432934492307464
119600000 21.50006690787287
119700000 20.806805289734722
119800000 20.084417999674947
119900000 20.998474929323198
120000000 10.955906844892036
120100000 2.798683583045135
120200000 19.450692645277552
120300000 20.3700615440989
120400000 21.00517327647722
120500000 20.685169420489842
120600000 21.906076727061798
120700000 21.46400512534797
120800000 22.070125387382706
120900000 22.690946317587557
121000000 21.659343055308057
121100000 18.26323513286985
121200000 18.000187470803752
121300000 16.749522153265705
121400000 17.5257734457

...

233000000 2.026892922747857
233100000 2.291166129452659
233200000 2.299082177416712
233300000 2.2812826794079513
233400000 2.1578490787206985
233500000 0.9765773214699128
233600000 0.529330211666521
233700000 2.175824941968693
233800000 2.924336785520265
233900000 2.0125493067800435
234000000 2.1533726006485288
234100000 2.0808276224956512
234200000 2.0523919868359766
234300000 2.302235232784288
234400000 2.000438657090853
234500000 2.271310576677753
234600000 2.433703531592935
234700000 1.9361157814867
234800000 2.044213246488548
234900000 2.1058879975150524
235000000 2.107194338109851
235100000 2.0493649710019994
235200000 2.245223967800548
235300000 1.9469383705710868
235400000 0.5223432687404752
235500000 0.5224199978558886
235600000 2.0815313019044397
235700000 1.9274777387300928
235800000 2.291765071464164
235900000 3.009193785017461
236000000 2.180936180957559
236100000 2.0937550999245182
236200000 2.1401219435782535
236300000 2.742672381207873
236400000 2.391585858295063
236500

261700000 3.1203050372480754
261800000 3.3228310303927393
261900000 3.20458584270263
262000000 3.1925052161968948
262100000 3.234108298034403
262200000 3.070681242685
262300000 3.1962637322031853
262400000 3.9021754372123816
262500000 4.040321942115857
262600000 3.469362487228622
262700000 3.269111492164582
262800000 3.2478905053997704
262900000 3.364069138550699
263000000 3.362192648229036
263100000 3.011414950738271
263200000 3.4732918145540363
263300000 0.7512686997656757
263400000 0.7676490808068746
263500000 2.8852468651756547
263600000 3.409046416261047
263700000 3.2733514795346657
263800000 3.5247446885970533
263900000 3.295369034730633
264000000 3.0597162403007876
264100000 3.3672108397123512
264200000 3.369405109971
264300000 4.499863156988498
264400000 4.2652402202275495
264500000 3.655338294591893
264600000 3.4631013644210995
264700000 3.796992356437951
264800000 3.863599136900945
264900000 3.543787050391366
265000000 3.2475288725613956
265100000 3.260437846717338
265200000 

290500000 2.17318782829238
290600000 2.199153499885761
290700000 2.027212266312914
290800000 2.273159847935468
290900000 2.1025988283722037
291000000 2.1289832461034095
291100000 2.1043620969238184
291200000 0.8407045612613658
291300000 0.5079616238762372
291400000 1.8022897218468785
291500000 1.9613511562724977
291600000 2.147471224460009
291700000 2.1969158641533855
291800000 2.2283028952253594
291900000 2.007247358291543
292000000 2.1857849215444127
292100000 2.3400555934777296
292200000 2.669032870416936
292300000 2.141663474314958
292400000 2.217306685606228
292500000 1.954214925911645
292600000 2.071134021864623
292700000 2.165641770796094
292800000 2.2665414390764136
292900000 1.9972271193893052
293000000 1.7825528616684245
293100000 0.5012473896633486
293200000 0.503216432288905
293300000 2.4527432207712168
293400000 2.013460357604732
293500000 2.114974341097176
293600000 2.088718483104368
293700000 2.2357768010651897
293800000 1.9313560058134893
293900000 2.251444098645369
294

318300000 1.213709736434016
318400000 1.2693743950730265
318500000 1.1296113908658656
318600000 1.404642862572127
318700000 1.242661383045733
318800000 1.2968072230200072
318900000 1.255408037933813
319000000 1.3029378853826525
319100000 1.1592991855179966
319200000 1.3252001591582996
319300000 1.167462903071652
319400000 1.2225345261946752
319500000 1.1898528909183475
319600000 1.1854072446836532
319700000 1.1762152915589048
319800000 1.3963573074203168
319900000 1.2371573499974906
320000000 1.2548265111263492
320100000 1.1663546525633632
320200000 1.135999864396443
320300000 1.2612839214909208
320400000 1.2529101631465422
320500000 1.2012208445172607
320600000 1.1735495795199475
320700000 1.137898338888963
320800000 1.0711933447313509
320900000 1.1427258818630013
321000000 1.1532331697842513
321100000 1.1385093692944648
321200000 1.1363590635335479
321300000 1.086981308052596
321400000 1.014332111886263
321500000 1.0472064870198865
321600000 1.108073767099599
321700000 1.089925865458

345200000 0.14267046067921652
345300000 0.21136042936508523
345400000 0.12353270173020164
345500000 0.142199285488728
345600000 0.1213969325155417
345700000 0.17087874410742523
345800000 0.15003337011131312
345900000 0.15171071662101482
346000000 0.1704044751372437
346100000 0.10495707920497988
346200000 0.15952962583845523
346300000 0.09451208467184505
346400000 0.14909779178929658
346500000 0.13993788626463213
346600000 0.09232622267231345
346700000 0.1186528594081468
346800000 0.09640765792825817
346900000 0.11362382625243068
347000000 0.10137209031992157
347100000 0.11203814766772588
347200000 0.1286067101791799
347300000 0.12715389403413402
best so far: 0
type: [1, 3, 1, 41, 1] 123
cases of this type: 206763
347400000 0.07710017811512615
347500000 0.0183982285771966
best so far: 0
type: [1, 3, 41, 1, 1] 123
cases of this type: 5043
best so far: 0
type: [1, 41, 1, 1, 3] 123
cases of this type: 3321
best so far: 0
type: [1, 41, 1, 3, 1] 123
cases of this type: 1107
best so far: 0
ty